Extraccion transformacion y carga de archivo "output_steam_games"

In [2]:
import pandas as pd
import json
import pickle


# from collections import Counter

In [6]:
# Leer datos de JSON e importar en dataframe
rows = []

with open('output_steam_games.json', encoding='MacRoman') as f:
    for line in f.readlines():
        data = json.loads(line)
        rows.append(data)

df_games = pd.DataFrame(rows)

In [7]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Información general del DataFrame, como tipos de datos y valores faltantes
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [9]:
# Comprobando si hay valores nulos en el DataFrame
df_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Con esta funcion descubrimos que todas las columnas tienen datos mixtos

In [10]:
# Definir una función para determinar el tipo de datos de una columna
def column_type(column):
    # Obtener los tipos de datos únicos en la columna
    unique_types = column.apply(type).unique()
    
    # Si hay un solo tipo de datos, devolver ese tipo
    if len(unique_types) == 1:
        return unique_types[0].__name__
    
    # Si hay varios tipos de datos, devolver 'mixed'
    return 'mixed'

# Aplicar la función a todas las columnas del DataFrame
result = df_games.apply(column_type, axis=0)

# Mostrar el resultado
print(result)


publisher       mixed
genres          mixed
app_name        mixed
title           mixed
url             mixed
release_date    mixed
tags            mixed
reviews_url     mixed
specs           mixed
price           mixed
early_access    mixed
id              mixed
developer       mixed
dtype: object


Reemplazamos todos los NaN por 'sin datos'

In [9]:
# Supongamos que tienes un DataFrame llamado 'df_games' y deseas reemplazar los NaN en todo el DataFrame
# df_juegos = df_games.fillna('sin datos', inplace=True)


In [11]:
df_juegos = df_games.dropna(how='all')

Reemplazamos todos los valores no ASCII por Unknown

In [13]:
def replace_non_ascii_strings(df, replacement_value="Unknown"):
    # Iterar sobre todas las columnas del DataFrame
    for column_name in df.columns:
        # Reemplazar los valores con caracteres no ASCII por el valor de reemplazo
        df[column_name] = df[column_name].apply(lambda x: replacement_value if isinstance(x, str) and any(ord(c) > 128 for c in x) else x)
    
    return df

# Llamar a la función para reemplazar caracteres no ASCII en todas las columnas
df_juegos = replace_non_ascii_strings(df_juegos, replacement_value="Unknown")


Procedemos averiguar que tipo de datos y que cantidad hay en cada columna

In [14]:
def count_column_types(df):
    # Aplicar la función a todas las columnas del DataFrame
    result = df.apply(lambda col: col.apply(type).value_counts())
    
    # Transponer el resultado y restablecer el índice
    result = result.T.reset_index()
    
    # Renombrar las columnas
    result.columns = ['Column'] + result.columns[1:].tolist()
    
    # Devolver el resultado
    return result

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)



          Column  <class 'float'>  <class 'str'>  <class 'list'>  \
0      publisher          96362.0        24083.0             NaN   
1         genres          91593.0            NaN         28852.0   
2       app_name          88312.0        32133.0             NaN   
3          title          90360.0        30085.0             NaN   
4            url          88310.0        32135.0             NaN   
5   release_date          90377.0        30068.0             NaN   
6           tags          88473.0            NaN         31972.0   
7    reviews_url          88312.0        32133.0             NaN   
8          specs          88980.0            NaN         31465.0   
9          price         118535.0         1910.0             NaN   
10  early_access          88310.0            NaN             NaN   
11            id          88312.0        32133.0             NaN   
12     developer          91609.0        28836.0             NaN   

    <class 'bool'>  
0              NaN  
1    

Pasamos todos los valores de release_date a fecha

In [16]:
# Convertir la columna 'release_date' al tipo datetime
df_juegos['release_date'] = pd.to_datetime(df_games['release_date'], format='%Y-%m-%d', errors='coerce')

# Formatear las fechas en "YYYY-MM-DD" sin la parte de la hora
df_juegos['release_date'] = df_games['release_date'].dt.strftime('%Y-%m-%d')




Cambiamos todos los valores flotantes a String dela columna release_date

In [17]:
df_juegos['release_date'] = df_games['release_date'].apply(lambda x: str(x) if isinstance(x, float) else x)


Verificamos valores de la columna release_date

In [13]:
# Define una función para verificar si una cadena está en formato de fecha
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d', errors='raise')
        return True
    except ValueError:
        return False

# Verifica si todos los valores en la columna 'release_date' son fechas válidas
all_dates_are_valid = df_games['release_date'].apply(is_valid_date).all()

if all_dates_are_valid:
    print("La columna 'release_date' está en formato de fecha válido.")
else:
    print("La columna 'release_date' no está en formato de fecha válido.")




La columna 'release_date' está en formato de fecha válido.


Verificamos la columna

In [18]:
df_juegos['release_date']

0                nan
1                nan
2                nan
3                nan
4                nan
             ...    
120440    2018-01-04
120441    2018-01-04
120442    2018-01-04
120443    2017-09-02
120444           nan
Name: release_date, Length: 120445, dtype: object

Pasamos los NaN a NaT (Valores No Válidos de Fecha y Hora).

In [19]:
# Convertir la columna 'release_date' al formato de fecha
df_juegos['release_date'] = pd.to_datetime(df_juegos['release_date'], format='%Y-%m-%d', errors='coerce')

# Reemplazar NaN en la columna 'release_date' con NaT (Valores No Válidos de Fecha y Hora)
df_juegos['release_date'].fillna(pd.NaT, inplace=True)


In [20]:
df_juegos['release_date']

0               NaT
1               NaT
2               NaT
3               NaT
4               NaT
            ...    
120440   2018-01-04
120441   2018-01-04
120442   2018-01-04
120443   2017-09-02
120444          NaT
Name: release_date, Length: 120445, dtype: datetime64[ns]

Imprimimos el df para verificar el tipo de datos

In [21]:
def count_column_types(df):
    # Aplicar la función a todas las columnas del DataFrame
    result = df.apply(lambda col: col.apply(type).value_counts())
    
    # Transponer el resultado y restablecer el índice
    result = result.T.reset_index()
    
    # Renombrar las columnas
    result.columns = ['Column'] + result.columns[1:].tolist()
    
    # Devolver el resultado
    return result

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)

          Column  <class 'float'>  <class 'str'>  <class 'list'>  \
0      publisher          96362.0        24083.0             NaN   
1         genres          91593.0            NaN         28852.0   
2       app_name          88312.0        32133.0             NaN   
3          title          90360.0        30085.0             NaN   
4            url          88310.0        32135.0             NaN   
5   release_date              NaN            NaN             NaN   
6           tags          88473.0            NaN         31972.0   
7    reviews_url          88312.0        32133.0             NaN   
8          specs          88980.0            NaN         31465.0   
9          price         118535.0         1910.0             NaN   
10  early_access          88310.0            NaN             NaN   
11            id          88312.0        32133.0             NaN   
12     developer          91609.0        28836.0             NaN   

    <class 'pandas._libs.tslibs.nattype.NaTType

Vemos los string de la columna price

In [22]:
# Filtrar las filas donde 'price' es de tipo cadena (string)
strings_en_price = df_juegos[df_juegos['price'].apply(lambda x: isinstance(x, str))]

# Mostrar los valores únicos de tipo cadena en la columna 'price'
unique_strings_en_price = strings_en_price['price'].unique()

# Mostrar los valores únicos
print(unique_strings_en_price)


['Free To Play' 'Free to Play' 'Free' 'Free Demo' 'Play for Free!'
 'Install Now' 'Play WARMACHINE: Tactics Demo' 'Free Mod' 'Install Theme'
 'Third-party' 'Play Now' 'Unknown' 'Play the Demo' 'Starting at $499.00'
 'Starting at $449.00' 'Free to Try' 'Free Movie' 'Free to Use']


Reemplazamos todos los string por 0

In [23]:
# Reemplazar todos los valores de tipo cadena en la columna 'price' por 0
df_juegos['price'] = df_juegos['price'].apply(lambda x: 0 if isinstance(x, str) else x)


Pasamos la columna price en su totalidad a valores que se puedan sumar

In [24]:
# Convierte la columna 'price' de df_games a tipo float, pero establece 'errors' en 'coerce' para manejar los valores no válidos como NaN
df_juegos['price'] = pd.to_numeric(df_juegos['price'], errors='coerce')

# Ahora puedes usar df como lo necesites


In [25]:
def count_column_types(df):
    # Aplicar la función a todas las columnas del DataFrame
    result = df.apply(lambda col: col.apply(type).value_counts())
    
    # Transponer el resultado y restablecer el índice
    result = result.T.reset_index()
    
    # Renombrar las columnas
    result.columns = ['Column'] + result.columns[1:].tolist()
    
    # Devolver el resultado
    return result

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)

          Column  <class 'float'>  <class 'str'>  <class 'list'>  \
0      publisher          96362.0        24083.0             NaN   
1         genres          91593.0            NaN         28852.0   
2       app_name          88312.0        32133.0             NaN   
3          title          90360.0        30085.0             NaN   
4            url          88310.0        32135.0             NaN   
5   release_date              NaN            NaN             NaN   
6           tags          88473.0            NaN         31972.0   
7    reviews_url          88312.0        32133.0             NaN   
8          specs          88980.0            NaN         31465.0   
9          price         120445.0            NaN             NaN   
10  early_access          88310.0            NaN             NaN   
11            id          88312.0        32133.0             NaN   
12     developer          91609.0        28836.0             NaN   

    <class 'pandas._libs.tslibs.nattype.NaTType

In [27]:
df_juegos['price']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
120440    1.99
120441    4.99
120442    1.99
120443    4.99
120444    4.99
Name: price, Length: 120445, dtype: float64

Reemplazar los valores NaN en la columna 'price' con 0

In [28]:

# Reemplazar los valores NaN en la columna 'price' con 0
df_juegos['price'].fillna(0, inplace=True)



In [29]:
def count_column_types(df):
    # Aplicar la función a todas las columnas del DataFrame
    result = df_games.apply(lambda col: col.apply(type).value_counts())
    
    # Transponer el resultado y restablecer el índice
    result = result.T.reset_index()
    
    # Renombrar las columnas
    result.columns = ['Column'] + result.columns[1:].tolist()
    
    # Devolver el resultado
    return result

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)


          Column  <class 'float'>  <class 'str'>  <class 'list'>  \
0      publisher          96362.0        24083.0             NaN   
1         genres          91593.0            NaN         28852.0   
2       app_name          88312.0        32133.0             NaN   
3          title          90360.0        30085.0             NaN   
4            url          88310.0        32135.0             NaN   
5   release_date              NaN            NaN             NaN   
6           tags          88473.0            NaN         31972.0   
7    reviews_url          88312.0        32133.0             NaN   
8          specs          88980.0            NaN         31465.0   
9          price         120445.0            NaN             NaN   
10  early_access          88310.0            NaN             NaN   
11            id          88312.0        32133.0             NaN   
12     developer          91609.0        28836.0             NaN   

    <class 'pandas._libs.tslibs.nattype.NaTType

Ver la columna price

In [30]:
df_juegos['price']


0         0.00
1         0.00
2         0.00
3         0.00
4         0.00
          ... 
120440    1.99
120441    4.99
120442    1.99
120443    4.99
120444    4.99
Name: price, Length: 120445, dtype: float64

Verificamos la columna price q esten en float

In [38]:
def count_column_types(df):
    # Aplicar la función a todas las columnas del DataFrame
    result = df_games.apply(lambda col: col.apply(type).value_counts())
    
    # Transponer el resultado y restablecer el índice
    result = result.T.reset_index()
    
    # Renombrar las columnas
    result.columns = ['Column'] + result.columns[1:].tolist()
    
    # Devolver el resultado
    return result

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)

          Column  <class 'float'>  <class 'str'>  \
0      publisher          96362.0        24083.0   
1         genres              NaN       120445.0   
2       app_name          88312.0        32133.0   
3          title          90360.0        30085.0   
4            url          88310.0        32135.0   
5   release_date              NaN            NaN   
6           tags          88473.0            NaN   
7    reviews_url          88312.0        32133.0   
8          specs          88980.0            NaN   
9          price         120445.0            NaN   
10  early_access          88310.0            NaN   
11            id          88312.0        32133.0   
12     developer          91609.0        28836.0   

    <class 'pandas._libs.tslibs.nattype.NaTType'>  \
0                                             NaN   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4     

Hacemos lo mismo con la columna Genres

In [39]:
def show_unique_values(df, column_name):
    # Obtener todas las listas de la columna especificada y combinarlas en una lista
    all_values = [value for sublist in df[column_name] if isinstance(sublist, list) for value in sublist]
    
    # Convertir los valores únicos en un conjunto para eliminar duplicados
    unique_values_set = set(all_values)
    
    # Convertir el conjunto nuevamente a una lista
    unique_values_list = list(unique_values_set)
    
    # Devolver el resultado
    return unique_values_list

# Llamar a la función para mostrar los valores únicos de la columna 'genres' del DataFrame 'df'
result = show_unique_values(df_juegos, 'genres')

# Mostrar el resultado
print(result)


[]


Ahora pasamos todos los valores de genres a String

In [40]:
def show_unique_values(df, column_name):
    # Convertir todos los valores de la columna a strings
    df[column_name] = df[column_name].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
    
    # Obtener los valores únicos de la columna especificada
    unique_values = df[column_name].unique()
    
    # Devolver el resultado
    return unique_values

# Llamar a la función para mostrar los valores únicos de la columna 'genres' del DataFrame 'df'
result = show_unique_values(df_juegos, 'genres')

# Mostrar el resultado
print(result)


['nan' 'Action, Casual, Indie, Simulation, Strategy'
 'Free to Play, Indie, RPG, Strategy'
 'Casual, Free to Play, Indie, Simulation, Sports'
 'Action, Adventure, Casual' 'Action, Adventure, Simulation'
 'Free to Play, Indie, Simulation, Sports'
 'Casual, Indie, Racing, Simulation'
 'Action, Indie, Simulation, Early Access'
 'Action, Adventure, Casual, Indie, RPG' 'Casual, Indie'
 'Casual, Indie, Simulation'
 'Adventure, Casual, Indie, Simulation, Strategy'
 'Action, Adventure, Indie' 'Racing, Simulation, Sports' 'Action, Indie'
 'Action, Indie, Racing' 'Action' 'Action, Indie, RPG'
 'Casual, Indie, Strategy'
 'Action, Casual, Indie, Massively Multiplayer, Simulation, Sports, Strategy'
 'Adventure, Casual, Indie' 'Action, Adventure, RPG'
 'Adventure, Casual, Indie, RPG' 'Strategy'
 'Indie, Massively Multiplayer, RPG, Early Access'
 'Adventure, Free to Play, RPG, Early Access'
 'Action, Indie, Strategy, Early Access'
 'Action, Adventure, Indie, RPG, Strategy' 'Action, Adventure, Indie, 

Verficamos que no haya listas en genres y que sean de tipo String

In [41]:
def count_column_types(df):
    # Aplicar la función a todas las columnas del DataFrame
    result = df_juegos.apply(lambda col: col.apply(type).value_counts())
    
    # Transponer el resultado y restablecer el índice
    result = result.T.reset_index()
    
    # Renombrar las columnas
    result.columns = ['Column'] + result.columns[1:].tolist()
    
    # Devolver el resultado
    return result

# Llamar a la función en un DataFrame 'df'
result = count_column_types(df_juegos)

# Mostrar el resultado
print(result)

          Column  <class 'float'>  <class 'str'>  \
0      publisher          96362.0        24083.0   
1         genres              NaN       120445.0   
2       app_name          88312.0        32133.0   
3          title          90360.0        30085.0   
4            url          88310.0        32135.0   
5   release_date              NaN            NaN   
6           tags          88473.0            NaN   
7    reviews_url          88312.0        32133.0   
8          specs          88980.0            NaN   
9          price         120445.0            NaN   
10  early_access          88310.0            NaN   
11            id          88312.0        32133.0   
12     developer          91609.0        28836.0   

    <class 'pandas._libs.tslibs.nattype.NaTType'>  \
0                                             NaN   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4     

Vemos el tipo de datos de la columna developer

In [44]:
# Obtener los valores únicos en la columna 'developer'
valores_developer = df_juegos['developer'].unique()

# Mostrar los valores únicos
print(valores_developer)


[nan 'Kotoshiro' 'Secret Level SRL' ... 'VersoVR' 'INGAME'
 'Bidoniera Games']


Vemos si contiene nan

In [46]:
# Verificar si la columna 'developer' contiene valores NaN
contiene_nan = df_juegos['developer'].isnull().any()
print("Contiene NaN:", contiene_nan)


Contiene NaN: True


Reemplazamos esos Nan por sin datos

In [48]:
# Reemplazar los valores NaN en la columna 'developer' por "Sin datos"
df_juegos['developer'].fillna("Sin datos", inplace=True)


Volvemo a verificar q no tenga Nan

In [49]:
# Verificar si la columna 'developer' contiene valores NaN
contiene_nan = df_juegos['developer'].isnull().any()
print("Contiene NaN:", contiene_nan)

Contiene NaN: False


Pasamos los valores float que tenia id a string

In [51]:
# Convertir los valores de tipo float en la columna 'id' a cadenas (strings)
df_juegos['id'] = df_juegos['id'].astype(str)

Verificamos que solo sea de numeros la columna id

In [54]:
import pandas as pd
import re

# Crear una función para verificar si una cadena contiene solo letras
def contiene_letras(cadena):
    return bool(re.search('[a-zA-Z]', cadena))

# Verificar si hay valores en la columna 'id' que contienen solo letras
valores_con_letras = df_juegos['id'].apply(lambda x: contiene_letras(x) if isinstance(x, str) and 'id' in x else False)

# Mostrar los valores que contienen letras en la columna 'id'
valores_letras_id = df_juegos[valores_con_letras]

# Mostrar los resultados
print("Valores en la columna 'id' que contienen letras:")
print(valores_letras_id)



Valores en la columna 'id' que contienen letras:
Empty DataFrame
Columns: [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
Index: []


Creamos una copia del df con las columnas limpias

In [55]:
# Crear una copia del DataFrame con las columnas específicas
df_games_copia = df_juegos[['genres', 'release_date', 'price', 'developer', 'id']].copy()

Eliminamos las filas que no tengan datos entre las 5 columnas limpias

In [66]:

# Convertir la columna 'price' a tipo numérico
df_games_copia['price'] = pd.to_numeric(df_games_copia['price'], errors='coerce')

# Llenar los valores NaN en la columna 'price' con 0.00
df_games_copia['price'].fillna(0.00, inplace=True)

# Eliminar las filas que cumplan con las condiciones especificadas
df_games_copia = df_games_copia[~((df_games_copia['developer'] == 'Sin datos') |
                                  (df_games_copia['release_date'].isna()) |
                                  (df_games_copia['genres'].isna()))]



In [67]:
df_games_copia

,genres,release_date,price,developer,id
88310,"Action, Casual, Indie, Simulation, Strategy",2018-01-04,4.99,Kotoshiro,761140
88311,"Free to Play, Indie, RPG, Strategy",2018-01-04,0.00,Secret Level SRL,643980
88312,"Casual, Free to Play, Indie, Simulation, Sports",2017-07-24,0.00,Poolians.com,670290
88313,"Action, Adventure, Casual",2017-12-07,0.99,Unknown,767400
88315,"Action, Adventure, Simulation",2018-01-04,3.99,Trickjump Games Ltd,772540
...,...,...,...,...,...
120439,"Action, Adventure, Casual, Indie",2018-01-04,1.99,Bidoniera Games,745400
120440,"Casual, Indie, Simulation, Strategy",2018-01-04,1.99,"Nikita ""Ghost_RUS""",773640
120441,"Casual, Indie, Strategy",2018-01-04,4.99,Sacada,733530
120442,"Indie, Racing, Simulation",2018-01-04,1.99,Laush Dmitriy Sergeevich,610660


In [68]:
df_games_copia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28533 entries, 88310 to 120443
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28533 non-null  object        
 1   release_date  28533 non-null  datetime64[ns]
 2   price         28533 non-null  float64       
 3   developer     28533 non-null  object        
 4   id            28533 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 1.3+ MB


Terminamos la limpieza

Lo guardamos como archivo parques

In [73]:
# Guardar df_games_copia en un archivo .pkl
with open('games.pkl', 'wb') as pkl_file:
    pickle.dump(df_games_copia, pkl_file)


# Probamos las funciones

In [3]:
# Importar el archivo 'reviews.pkl' en formato Parquet
df_reviews = pd.read_parquet('reviews.pkl')

# Importar el archivo 'items.pkl' en formato Parquet
df_items = pd.read_parquet('items.pkl')

# Importar el archivo 'games.pkl' en formato Parquet
df_games = pd.read_parquet('games.pkl')




ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [4]:
def userdata(user_id, df_reviews, df_items):
    # Filtrar las revisiones del usuario específico
    user_reviews = df_reviews[df_reviews['user_id'] == user_id]

    # Calcular la cantidad de dinero gastado por el usuario
    money_spent = user_reviews['price'].sum()

    # Calcular el porcentaje de recomendación basado en las revisiones
    total_reviews = len(user_reviews)
    recommended_reviews = user_reviews['recommend'].str.lower().eq('true').sum()
    
    if total_reviews > 0:
        recommendation_percentage = (recommended_reviews / total_reviews) * 100
    else:
        recommendation_percentage = 0

    # Calcular la cantidad de ítems que ha revisado
    items_reviewed = total_reviews

    # Devolver los resultados
    return {
        'money_spent': money_spent,
        'recommendation_percentage': recommendation_percentage,
        'items_reviewed': items_reviewed
    }

# Uso de la función para obtener datos del usuario
user_data = userdata('user_id_a_consultar', df_reviews, df_items)

# Imprimir los resultados
print(user_data)


KeyError: 'price'

In [ ]:
# Luego, llama a la función userdata con el ID del usuario que deseas consultar
user_data = userdata('76561197970982479', df_reviews, df_items)

# Imprime los resultados
print(user_data)

NameError: name 'df_reviews' is not defined

In [ ]:
def users_reviews_between_dates(merged_df, start_date, end_date):
    filtered_df = merge_df[(merged_df['posted'] == start_date) & (merged_df['posted'] == end_date)]
    total_users = filtered_df['user_id'].nunique()
    recommend_percentage = filtered_df['recommend_percentage']
    return total_users, recommend_percentage

In [ ]:
start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filtrar las reviews entre las fechas dadas
    filtered_reviews = reviews[(reviews['posted'] >= start_date) & (reviews['posted'] <= end_date)]
    
    # Calcular la cantidad de usuarios únicos que realizaron reviews en el período
    unique_users = filtered_reviews['user_id'].nunique()
    
    # Calcular el porcentaje de recomendación
    recommendation_percentage = (filtered_reviews['recommend'].sum() /

In [ ]:
# Lista de columnas relevantes
columnas_similares = ['genres', 'tags', 'specs']

# Función para combinar listas en una cadena
def combinar_listas(row):
    combined = ' '.join(item for sublist in row if sublist for item in sublist)
    return combined

# Crear una nueva columna "variables_juntas" con las listas combinadas en cadenas
df_sg_nec['variables_juntas'] = df_sg_nec[columnas_similares].apply(combinar_listas, axis=1)
# Crear una matriz TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_sg_nec['variables_juntas'])

In [ ]:
# Reducción de dimensionalidad con Truncated SVD
num_components = 50  # Número de componentes a mantener
svd = TruncatedSVD(n_components=num_components)
svd_matrix = svd.fit_transform(tfidf_matrix)
# Calcular la similitud de coseno en la matriz reducida
cosine_sim_svd = cosine_similarity(svd_matrix, svd_matrix)

In [ ]:
def sentiment_analysis(año: int):
    # Unir los DataFrames en base a una columna común
    df_merged = pd.merge(steam, reviews, left_on='item_id', right_on='item_id')

    # Convierte 'releasedate' a datetime y extrae el año
    df_merged['year'] = pd.to_datetime(df_merged['releasedate']).dt.year

    # Filtra el DataFrame por el año dado
    df_año = df_merged[df_merged['year'] == año]

    # Cuenta las ocurrencias de cada sentimiento
    conteo_sentimientos = df_año['sentiment_analysis'].valu